# HASH Table
## Что такое Hash Table?
Хеш таблица — это структура данных, которая хранит пары ключ-значение. Ключ в нашем случае это строка, а значение — это адрес в памяти, где хранится значение. По ключу можно узнать bucket, в котором хранится значение, однако при коллизии приходится дополнительно идти по списку bucketа, чтобы найти нужное значение. 


![hashTable_normalSituation](imgs/hashTable_normalSituation.svg)

Коллизия — это ситуация, когда два ключа имеют одинаковый хэш. В таком случае два слова могут попасть в один bucket. Пэтому bucket — это список, в котором хранятся все значения, имеющие одинаковый хэш. У меня bucket - это двусвязный список. Можно было сделать односвязный список, но в скорости удаления элемента он проигрывает.

![hashTable_collision](imgs/hashTable_collision.svg)

## Оптимизации
### 1. Переключение с `-O0` на `-O2`
<style>
.highlight-box {
    color:rgb(19, 124, 63); /* Зелёный цвет текста */
    font-weight: bold; /* Жирный шрифт */
}
.
.cols-container {
    display: flex; /* Располагаем элементы в строке */
    justify-content: space-between; /* Распределяем пространство между колонками */
    gap: 0px; /* Отступ между колонками */
    font-size: 12px;
}

.two-column {
    width: 48%; /* Ширина каждой колонки */
}

summary {
    color:rgb(79, 120, 197); /* Зелёный цвет текста */
}
</style>


Самая простая и очевидная оптимизация это включить оптимизацию компилятора `-O2`. Это позволяет компилятору использовать более эффективные алгоритмы.  
В таком случае получем следующую информацию сравнения:
<details>
<summary>Click to show comparison</summary>  

Первое число - `-O2`, второе число - `-O0`  

**Time**  

`Elapsed Time`:	11.069s - 14.746s = -3.677s  
`CPU Time`:	8.157s - 11.666s = -3.509s  

Hardware Events
    Hardware Event Type	Hardware Event Count	Hardware Event Sample Count	Events Per Sample	Precise
    cycles			4000	Not changed, False

**Hardware Events**  

| Hardware Event Type | Hardware Event Count  | Hardware Event Sample Count	| Events Per Sample | Precise |
|---------------------|-----------------------|-----------------------------|-------------------|---------|
|cycles	| 19,577,064,201 - 27,998,395,201 = -8,421,331,000 | 43,429 - 58,032 = -14,603 | 4000 | Not changed, False |

**Вкладка Caller/Calle**  

![compare-O2and-O0](imgs/compare-O2and-O0.png)

</details>  

<div class="highlight-box">Процессорное время уменьшилось на 30.1 %, что достаточно хороший результат.</div>

### 2. Добавление SIMD instructions
Из предыдущего пункта видно, что много тактов процессора уходит на вызов функции `strcmp`, которая находится в `searchHT`. Это можно исправить, если использовать SIMD instructions. Условимся на том, что каждое слова будет не более 32 байт. В таком случае можно обрабатывать слова ка вектор `_mm256i`.  

<div class="cols-container">
    <div class="two-column">

**Старая версия:**  
```c
    int searchHT(HashTable* table, const unsigned char* word)
    {
        assert(table);
        assert(word);
        
        size_t index = hashFunction(word);
        Node* current = table->buckets[index].head;
        while (current != NULL) 
        {
            if (strcmp(current->word, (const char*)word) == 0) 
            {
                return 1;
            }
            current = current->next;
        }
        return 0;
    }
```
</div> <div class="two-column">

**Новая версия:**  
```c
    int searchHT(HashTable* table, const unsigned char* word) 
    {
        assert(table);
        assert(word);

        size_t index = hashFunction(word);
        Node* current = table->buckets[index].head;
        // Загружаем 256 бит = 32 байт (1 слово) в вектор
        __m256i word_vec = _mm256_loadu_si256((const __m256i*)word);

        while (current != NULL) 
        {

            __m256i current_vec = _mm256_loadu_si256((const __m256i*)current->word);
            // Сравниваем слова
            __m256i cmp_result = _mm256_cmpeq_epi8(word_vec, current_vec);
            // Создаем битовую маску
            int mask = _mm256_movemask_epi8(cmp_result);

            if (mask == 0xFFFFFFFF) {
                return 1;
            }
            current = current->next;
        }
        return 0;
    }
```
</div> </div>

Также для лучшего распределения попробуем переписать базовую версию хеш функции на 32-битную хеш функцию `djb2` на 
ассемблер.

Распределение `djb2` явно лучше тупого сложения ascii символов. Это видно из следующих диаграм:
<div style="display: flex; justify-content: space-between; ">
    <div style="width: 49%; text-align: center;">
        <b> ASCII summary</b>
        <img src="imgs/easiestASCII_hash.png" style="width: 100%;">
    </div>
    <div style="width: 49%; text-align: center;">
        <b> DJB2 </b>
        <img src="imgs/djb2_hash.png" style="width: 100%;">
    </div>
</div>

<div class="cols-container">
<div class="two-column">

**Старая версия:**  

Новый хеш получается умножение старого на простое число 31 (для более хорошого хеширования) и сложением
 его с ASCII кодом текущего символа.
```c
static size_t hashFunction(const unsigned char* word)
{
    assert(word);
    unsigned long hash = 0;

    for (size_t i = 0; word[i] != '\0'; i++) 
    {
        hash = hash + (unsigned long)word[i];
    }
    return hash % c_tableSize;
}
```

</div> <div class="two-column">

**Новая версия:**  

Хеш вычисляется следующим образом:  
`hash = ((hash << 5) + hash) + *word`
```asm
section .text
global myDjb2

; djb2 hash func

myDjb2:
    
    push rbx
    mov ebx, 5381

.loop:
    
    movzx eax, byte [rdi]  
    test al, al  
    jz .done     

    mov ecx, ebx
    shl ecx, 5   ; hash << 5
    add ecx, ebx ; (hash << 5) + hash = hash * 33
    add ecx, eax ; + *word
    mov ebx, ecx
    
    inc rdi      
    jmp .loop    

.done:

    mov eax, ebx 
    xor edx, edx 
    mov ecx, 750         
    div ecx      
    mov eax, edx 

    pop rbx
    ret
```
</div> </div>

В таком случае получем следующую информацию сравнения:
<details>
<summary>Click to show comparison</summary>  

Первое число - `-O2 with AVX, my hash func djb2`, второе число - `-O2 no AVX, base hash func`  

**Time**  

`Elapsed Time:` 4.339s - 11.069s = -6.730s  
`CPU Time:`	7.150s - 8.157s = -1.008s  

**Hardware Events**  

| Hardware Event Type | Hardware Event Count  | Hardware Event Sample Count	| Events Per Sample | Precise |
|---------------------|-----------------------|-----------------------------|-------------------|---------|
|cycles	| 17,158,995,089 - 19,577,064,201 = -2,418,069,112 | 17,327 - 43,429 = -26,102 | 4000 | Not changed, False |

**Вкладка Caller/Calle**  

![compareONAVXandOFFAVX](imgs/AVXDJB2_vs_NOAVXBASEHASH.png)

</details>  

<div class="highlight-box">Процессорное время уменьшилось на 12.4 %</div>

### 3. Заменить ассеммблер на C реализацию.

Давайте теперь попробуем заменить ассемблерную реализацию хеш функции на C реализацию, чтобы компилятор 
мог сам применить оптимизации для данной функции.

```c
uint32_t myDjb2(const unsigned char* word) 
{
    uint32_t hash = 5381;
    while (*word) {
        hash = ((hash << 5) + hash) + *word;
        word++;
    }
    return hash % c_tableSize;
}
```

Реализация `-O2` лучше оптимизировала хеш функцию, в частности "тяжеловесное" деление (у меня это `div ecx`)
 заменилось на "легкие" инструкции.

![O2hashFuncVSmyHashFunc](imgs/O2hashFuncVSmyHashFunc.svg)



В таком случае получем следующую информацию сравнения с изначальной версией (**-O2 without AVX**):
<details>
<summary>Click to show comparison</summary>  

Первое число - `-O2 with AVX, compiler hash func djb2`, второе число - `-O2 no AVX, base hash func`  

**Time**  

`Elapsed Time`:	4.246s - 4.339s = -0.093s  
`CPU Time`:	6.976s - 7.150s = -0.174s  

**Hardware Events**  

| Hardware Event Type | Hardware Event Count  | Hardware Event Sample Count	| Events Per Sample | Precise |
|---------------------|-----------------------|-----------------------------|-------------------|---------|
|cycles	| 16,742,374,695 - 17,158,995,089 = -416,620,394 | 16,934 - 17,327 = -393 | 4000 | Not changed, False |

**Вкладка Caller/Calle**  

![AVXCompilerDJB2_AVXBaseHash](imgs/COMPILERDJB2_vs_MYASMDJB2.png)

</details>  
<div class="highlight-box">Процессорное время уменьшилось на 2.4 %</div>

Дальнейшие опыты проводились в учебных целях, особого прироста к скорости исполнения программы (скорость поиска элемента в хеш таблице в частности) они не дают. 

### 4. Меняем fgets + SIMD на fread + Inline asm

Как видно из последнего скриншота вкладки Caller/Calle, функция `fgets` занимает аж 2е место по колонке `Self`,
 которая показывает `hardware events` или такты на собственное выполнение, что достаточно много и затратно.
Раз уж мы сказали, что слова у нас 32 байта, то файл ключей будем заранее подготавливать, добавляя нули 
до 32 байт. Из файла будем читать с помощью `fread(buffer, 1, 32, file)`
![preparedKeys](imgs/preparedKeys.png)

В учебных целях для опыта работы с `inline asm` пеерепишем 3 AVX инструкции. Всю информацию можно найти на 
[Intel® Intrinsics Guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html).

#### Краткая информация по AVX инструкциям:

| AVX Function Name          | AVX Function Parameters                      | Return Value            | Brief Description of What It Does                     |
|----------------------------|---------------------------------------------|-------------------------|-------------------------------------------------------|
| `_mm256_loadu_pd`          | `const double* ptr`                         | `__m256d`               | Loads 4 `double` values from memory into an AVX register |
| `_mm256_cmpeq_epi8` | `__m256i a, __m256i b` | `__m256i` | Compares 32 pairs of 8-bit integers for equality |
| `_mm256_movemask_epi8` | `__m256i a` | `scalar bit mask (int)` | Converts 32 comparison results into a 32-bit mask |

#### SIMMD -> inline assembly

- _mm256_loadu_si256
```c
asm volatile (
            "vmovdqu (%1), %%ymm0\n\t"
            : "=v" (current_vec) 
            : "r" (current->word)
            : "memory"
        );
```

- _mm256_cmpeq_epi8
```c
asm volatile (
                "vpcmpeqb %1, %2, %0\n\t"
                : "=v" (cmp_result)
                : "v" (word_vec), "v" (current_vec)
                :
            );
```

- _mm256_movemask_epi8
```c
asm volatile (
            "vpmovmskb %1, %0\n\t"
            : "=r" (mask)
            : "v" (cmp_result)
            :
        );
```

В таком случае получем следующую информацию сравнения с изначальной версией (**-O2 without AVX**):
<details>
<summary>Click to show comparison</summary>  

Первое число - `change fgets to fread`, второе число - `with fgetsc`  

**Time**  

`Elapsed Time`:	4.094s - 4.246s = -0.152s  
`CPU Time`:	6.760s - 6.976s = -0.216s  

**Hardware Events**  

| Hardware Event Type | Hardware Event Count  | Hardware Event Sample Count	| Events Per Sample | Precise |
|---------------------|-----------------------|-----------------------------|-------------------|---------|
|cycles	| 16,223,629,401 - 16,742,374,695 = -518,745,294 | 16,332 - 16,934 = -602 | 4000 | Not changed, False |

**Вкладка Caller/Calle**  

![INLINEASMFREAD_vs_SIMDFGETS](imgs/INLINEASMFREAD_vs_SIMDFGETS.png)

</details>  
<div class="highlight-box">Процессорное время уменьшилось на 3.1%</div>

## Вывод

В ходе работы были исследованы различные способы оптимизации. Были проведены тесты на скорость выполнения программы.
Было показано насколько сильно ускоряет `-O2` оптимизация и `SIMD` инструкции. Была проведена работа с 
ассемблерным кодом и ассемблерной инлайн вставкой.

## Инструменты и полезные материалы
- [Hotspot](https://github.com/KDAB/hotspot)
- [Intel Vtune](https://www.intel.com/content/www/us/en/developer/tools/oneapi/vtune-profiler.html)
- [Intel® Intrinsics Guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html)
- [Godbolt](godbolt.org/)
- [Jupiter Notebook](https://jupyter.org/)
- [VS Code](https://code.visualstudio.com/)
